<a href="https://colab.research.google.com/github/Sirabhop/Kaggle_Challenge/blob/master/Challenge_1_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import os

In [116]:
os.chdir("/content/drive/My Drive/My Project/Kaggle/Challenge#1_Titanic")

In [118]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  889 non-null    object  
 1   Survived     889 non-null    category
 2   Pclass       889 non-null    category
 3   Sex          889 non-null    category
 4   Age          889 non-null    float64 
 5   SibSp        889 non-null    int64   
 6   Parch        889 non-null    int64   
 7   Ticket       889 non-null    object  
 8   Fare         889 non-null    float64 
 9   Embarked     889 non-null    category
dtypes: category(4), float64(2), int64(2), object(2)
memory usage: 45.7+ KB


In [120]:
data['PassengerId'] = data['PassengerId'].astype('object')
data['Survived'] = data['Survived'].astype('category') #0 is no, 1 is yes
data['Pclass'] = data['Pclass'].astype('category')
data['Pclass'].cat.reorder_categories([1, 2, 3], ordered = True)
data['Sex'] = data['Sex'].astype('category')
data['Embarked'] = data['Embarked'].astype('category')

In [121]:
data.drop(['Name', 'Cabin'], axis = 1, inplace = True)
data.dropna(subset = ['Embarked'], axis = 0, how = 'any', inplace = True)
data['Age'].fillna(data['Age'].mean(), inplace = True)

data.reset_index(inplace = True, drop = True)

The question is who gonna survive from shipwreak? from those features I can draw 3 assumption


1. **The number of sibling, children, & parent** are effected the chance of survive because the titanic passenger would do ever it take to save their relatives.
2. **The room** where passenger stayed affected too, the chance of survival may depend on how easy for them to acquire the emergency boat.
3. **The status of their ticket**, one who are VIP guest on board may thought the crew cabin would rushed out to help them instead of economics passengers.





In [131]:
X = data.iloc[:, [0,2,3,4,5,6,7,8,9]]
y = data.iloc[:, 1]

In [128]:
X_train, X_test = train_test_split(X, test_size = 0.2, random_state = 0)
y_train, y_test = train_test_split(y, test_size = 0.2, random_state = 0)

In [ ]:
classifier = SVC(kernel = 'rbf')
classifier.fit(X_train, y_train)